# P2 MAAAI 

PREPROCESADO DE DATOS:

In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import numpy as np

In [7]:

# Cargar el conjunto de datos CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

print(f"Entrenamiento: {x_train.shape}, Prueba: {x_test.shape}")

# Eliminamos el 80% de las etiquetas en el conjunto de entrenamiento
np.random.seed(42)
unlabeled_indices = np.random.choice(len(x_train), size=int(0.8 * len(x_train)), replace=False)
labeled_indices = np.setdiff1d(np.arange(len(x_train)), unlabeled_indices)

# Creamos las particiones de datos
x_train_labeled = x_train[labeled_indices]
y_train_labeled = y_train[labeled_indices]
x_train_unlabeled = x_train[unlabeled_indices]
y_train_unlabeled = y_train[unlabeled_indices]  

print(f"Instancias de entrenamiento etiquetadas: {x_train_labeled.shape}, No etiquetadas: {x_train_unlabeled.shape}")


Entrenamiento: (50000, 32, 32, 3), Prueba: (10000, 32, 32, 3)
Instancias de entrenamiento etiquetadas: (10000, 32, 32, 3), No etiquetadas: (40000, 32, 32, 3)


CREACIÓN DE LA RED NEURONAL CONVOLUCIONAL

In [25]:
from tensorflow.keras import layers, models

In [42]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

def crear_modelo():
    model = models.Sequential()
    
    # Capa convolucional 1
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.0001), input_shape=(32, 32, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Capa convolucional 2
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Capa convolucional 3
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Capa convolucional 4
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Aplanamos las salidas
    model.add(layers.Flatten())
    
    # Capa densa 1
    model.add(layers.Dense(1000, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    
    # Capa densa 2
    model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    
    # Capa de salida
    model.add(layers.Dense(100, activation='softmax'))
    
    # Compilamos el modelo
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model = crear_modelo()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [43]:
#ENTRENAMIENTO
history = model.fit(x_train_labeled, y_train_labeled, epochs=30, batch_size=256)

Epoch 1/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 44s 958ms/step - accuracy: 0.0587 - loss: 8.3293
Epoch 2/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 37s 918ms/step - accuracy: 0.1102 - loss: 7.2294
Epoch 3/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 38s 952ms/step - accuracy: 0.1540 - loss: 5.6007
Epoch 4/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.1685 - loss: 5.1996
Epoch 5/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 35s 885ms/step - accuracy: 0.1940 - loss: 5.0825
Epoch 6/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 34s 844ms/step - accuracy: 0.2321 - loss: 4.7258
Epoch 7/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 40s 998ms/step - accuracy: 0.2531 - loss: 4.6720
Epoch 8/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.2453 - loss: 4.7905
Epoch 9/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 34s 857ms/step - accuracy: 0.2776 - loss: 4.8511
Epoch 10/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.3145 - loss: 4.6997
Epoch 11/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.3413 - loss: 4.4936
Epoch 12/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - 

In [44]:
#CONJUNTO DE PRUEBA
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

print(f"Rendimiento en la prueba: Pérdida = {test_loss}, Precisión = {test_acc}")

313/313 - 18s - 56ms/step - accuracy: 0.2600 - loss: 6.0727
Rendimiento en la prueba: Pérdida = 6.0727033615112305, Precisión = 0.25999999046325684


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def crear_modelo():
    model = models.Sequential()

    # Bloque convolucional 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.00005), input_shape=(32, 32, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Bloque convolucional 2
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.00005)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Bloque convolucional 3
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.00005)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Aplanar
    model.add(layers.Flatten())

    # Bloque denso 1
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.6))

    # Capa de salida
    model.add(layers.Dense(100, activation='softmax'))

    # Compilación del modelo
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


# Crear modelo
model_mejorado = crear_modelo()

# Callbacks para mejorar el entrenamiento
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
]

# Entrenamiento del modelo
history = model_mejorado.fit(
    x_train_labeled, y_train_labeled,
    epochs=20,
    batch_size=128,
    validation_data=(x_test, y_test),
    callbacks=callbacks
)




Epoch 1/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 29s 342ms/step - accuracy: 0.0339 - loss: 5.3104 - val_accuracy: 0.0191 - val_loss: 30.1947 - learning_rate: 0.0100
Epoch 2/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 343ms/step - accuracy: 0.1002 - loss: 4.4357 - val_accuracy: 0.0451 - val_loss: 8.0438 - learning_rate: 0.0100
Epoch 3/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 337ms/step - accuracy: 0.1301 - loss: 4.0582 - val_accuracy: 0.0639 - val_loss: 6.2281 - learning_rate: 0.0100
Epoch 4/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 353ms/step - accuracy: 0.1669 - loss: 3.8037 - val_accuracy: 0.0779 - val_loss: 5.6539 - learning_rate: 0.0100
Epoch 5/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 340ms/step - accuracy: 0.1992 - loss: 3.5936 - val_accuracy: 0.1385 - val_loss: 4.2949 - learning_rate: 0.0100
Epoch 6/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 359ms/step - accuracy: 0.2503 - loss: 3.3439 - val_accuracy: 0.1757 - val_loss: 3.8080 - learning_rate: 0.0100
Epoch 7/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 363ms/step - accuracy: 0.2710 - loss: 3

In [28]:
# Evaluamos el rendimiento en el conjunto de prueba
train_loss, train_acc = model_mejorado.evaluate(x_train_labeled, y_train_labeled, verbose=0)
test_loss, test_acc = model_mejorado.evaluate(x_test, y_test, verbose=2)
print("Self-training: Train Accuracy: {:.4f} - Test Accuracy: {:.4f}".format(train_acc, test_acc))


313/313 - 3s - 8ms/step - accuracy: 0.3130 - loss: 3.5625
Self-training: Train Accuracy: 0.7486 - Test Accuracy: 0.3130
